# Анализ источников новостей

Цель данного исследования: проанализировать источники новостных лент, сравнить рубрики, подумать над возможностью добавление\удаление категорий для проекта "Радар новостей рунета"

In [1]:
import numpy as np
import pandas as pd
import datetime as DT

## Interfax

Загрузим датасет interfax. Одновременно сделаем преобразование даты из строки

In [3]:
dateparse = lambda x: pd.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

In [5]:
data_interfax = pd.read_csv('../data/interfax.csv',
                      sep=',',
                      index_col=False,
                      escapechar='\\',
                      parse_dates=['date'],
                      date_parser=dateparse,
                      quotechar='\"',
                      error_bad_lines=False)

b'Skipping line 158938: expected 17 fields, saw 20\n'
b'Skipping line 176941: expected 17 fields, saw 20\n'
b'Skipping line 280155: expected 17 fields, saw 23\n'


In [6]:
# Проверим преобразование
data_interfax.dtypes

date           datetime64[ns]
url                    object
edition                object
topics                 object
authors                object
title                  object
text                   object
reposts_fb             object
reposts_vk             object
reposts_ok             object
reposts_twi            object
reposts_lj             object
reposts_tg             object
likes                  object
views                  object
comm_count             object
dtype: object

In [7]:
data_interfax.shape

(543961, 16)

In [8]:
# Так выглядет датасет interfax
data_interfax.head()

,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2019-04-30 13:26:00,https://www.sport-interfax.ru/659837,www.sport-interfax.ru,Спорт,NaN,Российского чемпиона мира по греко-римской бор...,Москва. 30 апреля. INTERFAX.RU - Чемпион мира ...,-,-,-,-,-,-,-,-,-
1,2019-05-07 12:46:00,https://www.sport-interfax.ru/660425,www.sport-interfax.ru,Спорт,NaN,Первый канал покажет все матчи россиян на ЧМ п...,Москва. 7 мая. INTERFAX.RU - Первый канал объя...,-,-,-,-,-,-,-,-,-
2,2019-05-08 12:57:00,https://www.sport-interfax.ru/660543,www.sport-interfax.ru,Спорт,NaN,Семак выразил надежду на новый контракт Кокори...,Москва. 8 мая. INTERFAX.RU - Главный тренер пе...,-,-,-,-,-,-,-,-,-
3,2019-05-09 00:06:00,https://www.sport-interfax.ru/660618,www.sport-interfax.ru,Спорт,NaN,"""Тоттенхэм"" стал вторым финалистом Лиги чемпионов",В финале он сыграет с другим представителем Ан...,-,-,-,-,-,-,-,-,-
4,2019-05-14 14:32:00,https://www.sport-interfax.ru/661091,www.sport-interfax.ru,Спорт,NaN,Претендовавшего на участие в ОИ скейтбордиста ...,Москва. 14 мая. INTERFAX.RU - Российский скейт...,-,-,-,-,-,-,-,-,-


In [9]:
# Выясним в каким столбцах есть пустые данные
data_interfax.isnull().any()

date           False
url            False
edition        False
topics         False
authors         True
title          False
text           False
reposts_fb     False
reposts_vk     False
reposts_ok     False
reposts_twi    False
reposts_lj     False
reposts_tg     False
likes          False
views          False
comm_count     False
dtype: bool

Как видим, в некоторых столбцах вместо значения Nan используется "-", их преобразованием займемся позже. А пока посмотрим строки, в которых пустые тексты новостей

In [11]:
print('Кол-во строк с значением NaN в столбце "authors" = ', data_interfax['authors'].isnull().sum())
data_interfax[data_interfax['authors'].isnull()]

Кол-во строк с значением NaN в столбце "authors" =  521464


,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2019-04-30 13:26:00,https://www.sport-interfax.ru/659837,www.sport-interfax.ru,Спорт,NaN,Российского чемпиона мира по греко-римской бор...,Москва. 30 апреля. INTERFAX.RU - Чемпион мира ...,-,-,-,-,-,-,-,-,-
1,2019-05-07 12:46:00,https://www.sport-interfax.ru/660425,www.sport-interfax.ru,Спорт,NaN,Первый канал покажет все матчи россиян на ЧМ п...,Москва. 7 мая. INTERFAX.RU - Первый канал объя...,-,-,-,-,-,-,-,-,-
2,2019-05-08 12:57:00,https://www.sport-interfax.ru/660543,www.sport-interfax.ru,Спорт,NaN,Семак выразил надежду на новый контракт Кокори...,Москва. 8 мая. INTERFAX.RU - Главный тренер пе...,-,-,-,-,-,-,-,-,-
3,2019-05-09 00:06:00,https://www.sport-interfax.ru/660618,www.sport-interfax.ru,Спорт,NaN,"""Тоттенхэм"" стал вторым финалистом Лиги чемпионов",В финале он сыграет с другим представителем Ан...,-,-,-,-,-,-,-,-,-
4,2019-05-14 14:32:00,https://www.sport-interfax.ru/661091,www.sport-interfax.ru,Спорт,NaN,Претендовавшего на участие в ОИ скейтбордиста ...,Москва. 14 мая. INTERFAX.RU - Российский скейт...,-,-,-,-,-,-,-,-,-
5,2019-05-22 14:06:00,https://www.sport-interfax.ru/662052,www.sport-interfax.ru,Спорт,NaN,Министр спорта призвал хоккеистов сборной Росс...,Москва. 22 мая. INTERFAX.RU - Тренерскому штаб...,-,-,-,-,-,-,-,-,-
6,2019-03-13 14:53:00,https://www.sport-interfax.ru/rfpl/654057,www.sport-interfax.ru,Спорт,NaN,"Защитник ""Спартака"" Жиго приступил к занятиям ...",Москва. 13 марта. INTERFAX.RU - Французский за...,-,-,-,-,-,-,-,-,-
7,2019-01-31 15:36:00,https://www.sport-interfax.ru/648598,www.sport-interfax.ru,Спорт,NaN,"Полузащитник ""Барселоны"" Денис Суарес перешел ...",Москва. 31 января. INTERFAX.RU - Лондонский фу...,-,-,-,-,-,-,-,-,-
8,2019-02-08 14:05:00,https://www.sport-interfax.ru/649781,www.sport-interfax.ru,Спорт,NaN,Сменился соперник чемпиона ОИ-2016 в боксе Тищ...,Москва. 8 февраля. INTERFAX.RU - Сменился сопе...,-,-,-,-,-,-,-,-,-
9,2019-02-21 15:28:00,https://www.sport-interfax.ru/651525,www.sport-interfax.ru,Спорт,NaN,МОК рассмотрит возможность включения брейк-дан...,Москва. 21 февраля. INTERFAX.RU - Оргкомитет л...,-,-,-,-,-,-,-,-,-


In [13]:
print('authors: ',data_interfax['authors'].unique())

authors:  [nan 'Анна Нехаева' 'Наталия Богданова' '/Интерфакс, Алексей Козин/'
 '/Интерфакс/' '/Интерфакс, Алиса Чудакова/' '/Анатолий Гусев, Интерфакс/'
 '/Алексей Козин, Интерфакс/' '/Интерфакс, Иван Баранов/'
 '/Интерфакс, Анатолий Гусев/' '/Иван Баранов, Интерфакс/'
 '/Карина Мелкумян, Интерфакс/' '/Алиса Чудакова, Интерфакс/'
 '/Интерфакс, Карина Мелкумян/' '/Интерфакс, Владимир Кондратьев/'
 '/Интерфакс, Дарья Карпенко/' '/Интерфакс, Зауреш Данаева/'
 '/Дарья Карпенко, Интерфакс/' '/Владимир Кондратьев, Интерфакс/'
 '/Финмаркет, Интерфакс/' '/Интерфакс, Алиса Чудакова, Анатолий Гусев/'
 '/Финмаркет/' '/Алор, Интерфакс/' '/Зауреш Данаева, Интерфакс/'
 '/Алексей Козин/' '/Интерфакс, Владимир Кондратьев, Дарья Карпенко/'
 '/Интерфакс, Дарья Карпенко, Анатолий Гусев/'
 '/Интерфакс, Владимир Кондратьев, Анатолий Гусев/' '/IFX.RU/'
 '/Финмаркет, IFX.RU/' '/Инфобизнес/' '/GlobalStock, IFX.RU/'
 '/Интерфакс, IFX.RU/' '/Интерфакс, Финмаркет/' '/Росинвестпроект/'
 '/Борис Грозовский, Дмитр

Посмотрим, что в столбцах, которые содержат дефис, как обозначение пустых данных

In [12]:
print('edition: ',data_interfax['edition'].unique())
print('reposts_fb: ', data_interfax['reposts_fb'].unique())
print('reposts_vk: ',data_interfax['reposts_vk'].unique())
print('reposts_ok: ',data_interfax['reposts_ok'].unique())
print('reposts_twi: ',data_interfax['reposts_twi'].unique())
print('reposts_lj: ',data_interfax['reposts_lj'].unique())
print('reposts_tg: ',data_interfax['reposts_tg'].unique())
print('likes: ',data_interfax['likes'].unique())
print('views: ',data_interfax['views'].unique())
print('comm_count: ',data_interfax['comm_count'].unique())

edition:  ['www.sport-interfax.ru' '-']
reposts_fb:  ['-']
reposts_vk:  ['-']
reposts_ok:  ['-']
reposts_twi:  ['-']
reposts_lj:  ['-']
reposts_tg:  ['-']
likes:  ['-']
views:  ['-']
comm_count:  ['-']


### Рубрики interfax

In [16]:
# Считаем количество рубрик в данном датасете, а также выведем значения
print('Количество рубрик = ', data_interfax.topics.unique().shape)
data_interfax['topics'].unique()

Количество рубрик =  (8,)


array(['Спорт', 'В мире', 'В России', 'Москва', 'Экономика', 'Культура',
       'Интервью', 'Пресс-центр'], dtype=object)

Все рубрики, с количеством новостей в каждой группе

In [15]:
data_interfax.groupby('topics')['topics'].count()

topics
В России       286148
В мире         102958
Интервью          583
Культура        12763
Москва          18426
Пресс-центр        29
Спорт           42994
Экономика       80060
Name: topics, dtype: int64